In [3]:
# --- СКРИПТ ДЛЯ SBER GIGACHAT (С ИСПРАВЛЕНИЕМ РАЗМЕРА ИЗОБРАЖЕНИЙ) ---

import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import requests
import base64
import uuid
import json
from PIL import Image # Импортируем библиотеку для сжатия
import io

# --- 1. Загрузка ключа и функция для получения токена ---
load_dotenv()
GIGACHAT_AUTH_DATA = os.getenv("GIGACHAT_AUTH_DATA")

def get_gigachat_token():
    if not GIGACHAT_AUTH_DATA: return None
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json', 'RqUID': str(uuid.uuid4()), 'Authorization': f'Basic {GIGACHAT_AUTH_DATA}'}
    payload = 'scope=GIGACHAT_API_PERS'
    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        response.raise_for_status()
        return response.json()['access_token']
    except requests.exceptions.RequestException as e:
        print(f"❌ Ошибка получения токена GigaChat: {e}")
        return None

# --- 2. Конфигурация и НОВАЯ ФУНКЦИЯ сжатия ---
IMG_DIR = "img3"
PROMPT = "Скажи, изображена ли на данном фото кошка породы \"Британская короткошерстная\", дай коротки ответ \"да\" или \"нет\". Опиши одним словом каждый признак, который убедил тебя в правильности или неправильности ответа. Если это не британская короткошерстная кошка, скажи какая это порода."
GIGACHAT_RESULTS_FILE = "gigachat_vision_results_v3.csv"
MODEL_GIGACHAT = "GigaChat:latest"

# --- НОВАЯ ФУНКЦИЯ: Изменение размера изображения ---
def resize_image(image_path, max_size=(1024, 1024)):
    """Открывает изображение, изменяет его размер и возвращает в виде байтов."""
    with Image.open(image_path) as img:
        img.thumbnail(max_size)
        byte_arr = io.BytesIO()
        if img.mode in ("RGBA", "P"): img = img.convert("RGB")
        img.save(byte_arr, format='JPEG')
        return byte_arr.getvalue()

# --- 3. Функция запроса к GigaChat API (ОБНОВЛЕННАЯ) ---
def query_gigachat_vision(image_path, prompt, access_token):
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
    try:
        # <-- НОВЫЙ ШАГ: Сжимаем изображение перед кодированием
        resized_image_bytes = resize_image(image_path)
        base64_image = base64.b64encode(resized_image_bytes).decode('utf-8')
        
        payload = {"model": MODEL_GIGACHAT, "messages": [{"role": "user", "content": f'<img src="data:image/jpeg;base64,{base64_image}" /> {prompt}'}], "temperature": 0.7, "max_tokens": 100}
        response = requests.post(url, headers=headers, json=payload, verify=False, timeout=60)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error: {type(e).__name__}, Details: {e}"

# --- 4. Основной цикл ---
print("Получаем токен доступа для GigaChat...")
access_token = get_gigachat_token()

if access_token:
    print("✅ Токен успешно получен. Начинаем работу.")
    try:
        image_files = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))])
        results_data = []
        for filename in tqdm(image_files, desc="GigaChat Processing"):
            image_path = os.path.join(IMG_DIR, filename)
            response = query_gigachat_vision(image_path, PROMPT, access_token)
            results_data.append({"filename": filename, "gigachat_response": response})
        
        df_results = pd.DataFrame(results_data)
        df_results.to_csv(GIGACHAT_RESULTS_FILE, index=False, encoding='utf-8')
        print(f"✅ Результаты GigaChat сохранены в '{GIGACHAT_RESULTS_FILE}'.")
        display(df_results.head())
    except FileNotFoundError:
        print(f"❌ Ошибка: Директория '{IMG_DIR}' не найдена.")
else:
    print("❌ Основной процесс не был запущен из-за ошибки получения токена.")

Получаем токен доступа для GigaChat...


c:\Users\Матвей\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Токен успешно получен. Начинаем работу.


GigaChat Processing:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\Матвей\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
GigaChat Processing:  10%|█         | 1/10 [00:00<00:07,  1.27it/s]c:\Users\Матвей\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
GigaChat Processing:  20%|██        | 2/10 [00:00<00:03,  2.24it/s]c:\Users\Матвей\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectio

✅ Результаты GigaChat сохранены в 'gigachat_vision_results_v3.csv'.


,filename,gigachat_response
0,rus1.jpg,"Error: HTTPError, Details: 402 Client Error: P..."
1,rus2.jpg,"Error: HTTPError, Details: 402 Client Error: P..."
2,rus3.jpg,"Error: HTTPError, Details: 402 Client Error: P..."
3,rus4.jpg,"Error: HTTPError, Details: 402 Client Error: P..."
4,rus5.jpg,"Error: HTTPError, Details: 402 Client Error: P..."
